In [1]:
import numpy as np

In [2]:
class Node:
    def __init__(self, sigma, parent=None):
        self.sigma = sigma
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
    
    def add_child(self, child):
        self.children.append(child)

    def update(self, reward):
        self.value += reward
        self.visits += 1

    def ucb1(self, total_visits, C=1.41):
        if self.visits == 0:
            return float('inf')  # Prioritize unvisited nodes
        return self.value / self.visits + C * np.sqrt(2 * np.log(total_visits) / self.visits)

In [3]:
def target_pdf(x):
    return np.exp(-x**2 / 2) / np.sqrt(2 * np.pi)

def mcmc_step(x_current, sigma):
    x_proposal = np.random.normal(x_current, sigma)
    accept_ratio = target_pdf(x_proposal) / target_pdf(x_current)
    if np.random.rand() < accept_ratio:
        return x_proposal, True
    return x_current, False

def best_child(node, C=1.41):
    choices_weights = [child.ucb1(node.visits, C) for child in node.children]
    return node.children[np.argmax(choices_weights)]


In [4]:
def mcts_simulation_sjd_with_chain(node, x_init=0, simulations=5_000):
    x_current = x_init # Initialize the current state
    sjd_total = 0  # Sum of squared jump distances
    mcmc_chain = [x_current]  # Store the MCMC chain
    nacc = 0

    for _ in range(simulations):
        x_new, is_accepted = mcmc_step(x_current, node.sigma)
        if is_accepted:
            sjd = (x_new - x_current) ** 2  # Calculate the squared jump distance
            sjd_total += sjd
            x_current = x_new  # Update the current state
            nacc += 1
        mcmc_chain.append(x_current)  # Store the current state in the MCMC chain
    sjd_average = sjd_total / simulations  # Calculate the average squared jump distance
    acc = nacc / simulations
    return sjd_average, mcmc_chain, acc

def mcts_sjd_with_chain(root, iterations=50, C=1.41, sigma_range=np.linspace(0.1, 30, 1000)):
    best_chain = []
    best_sjd = 0
    best_sigma = 0

    for _ in range(iterations):
        node = root
        # Choose
        while node.children:
            node = best_child(node, C)

        # Expand
        new_sigma = np.random.choice(sigma_range)
        new_node = Node(new_sigma, parent=node)
        node.add_child(new_node)

        # Simulate
        reward, chain, acc = mcts_simulation_sjd_with_chain(new_node)

        # Update the best chain
        if reward > best_sjd:
            best_sjd = reward
            best_chain = chain
            best_sigma = new_sigma
            best_acc = acc

        # Backpropagate
        while new_node:
            new_node.update(reward)
            new_node = new_node.parent

    return best_sigma, best_sjd, best_chain, best_acc

# Initialize the root node
root_chain = Node(1.0) # The initial sigma value is 1.0
best_sigma_chain, best_sjd_chain, best_chain, best_acc = mcts_sjd_with_chain(root_chain)

best_sigma_chain, best_sjd_chain, best_acc, best_chain[-1]


(2.703903903903904, 0.8215258210849341, 0.416, 1.8698680366540021)